In [2]:

# Import necessary libraries
import boto3
import requests
import json

# Function to get GitHub token from AWS Secrets Manager
def get_github_token(secret_name):
    # Create a Secrets Manager client
    client = boto3.client('secretsmanager')
    
    # Retrieve the secret value
    response = client.get_secret_value(SecretId=secret_name)
    secret = response['SecretString']
    secret_dict = json.loads(secret)
    return secret_dict['github-token1']

# Function to create a GitHub webhook
def create_github_webhook(repo_owner, repo_name, token, webhook_url):
    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/hooks"
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    payload = {
        "name": "web",
        "active": True,
        "events": ["push"],
        "config": {
            "url": webhook_url,
            "content_type": "json"
        }
    }
    response = requests.post(url, headers=headers, json=payload)
    return response

# Define parameters
secret_name = "github-token1"  # Name of the secret in AWS Secrets Manager
repo_owner = "chirujlns"       # GitHub username
repo_name = "nyc-taxi-e2e"     # Repository name
webhook_url = "https://example.com/webhook"  # URL for the webhook

# Get GitHub token from AWS Secrets Manager
github_token = get_github_token(secret_name)

# Create GitHub webhook
response = create_github_webhook(repo_owner, repo_name, github_token, webhook_url)

# Print response
print(response.status_code)
print(response.json())